In [1]:
import random
import numpy as np
import torch
import os
import pathlib
import pickle
from gurobipy import *
from rsome import ro
from rsome import grb_solver as grb
import rsome as rso
from rsome import cpt_solver as cpt

In [2]:
# data_generation_process = "SPO_Data_Generation"
data_generation_process = "DDR_Data_Generation"

# Parameters

In [ ]:
# import pyepo
# generate data
grid = (2,2) # grid size
num_train = 100 # number of training data
num_feat = 5 # size of feature
num_test = 1000
deg = 1.0 # polynomial degree
e = 1 # scale of normal std or the range of uniform. For the error term

lower = 0 # coef lower bound
upper = 1 # coef upper bound
p = num_feat # num of features
d = (grid[0] - 1) * (grid[0] - 1) * 2 + 2 * (grid[0] - 1) # num of arcs
num_nodes = grid[0]*grid[0]
alpha = e # scale of normal std or the range of uniform. For the error term
mis = deg # model misspecification
coef_seed = 1

x_dist = 'uniform'
e_dist = 'normal'
x_low = -2
x_up = 2
x_mean = 2
x_var = 2
bump = 7

In [ ]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
grandparent_directory = os.path.dirname(parent_directory)
DataPath = os.path.dirname(grandparent_directory) + '/Data/Test0401_' + data_generation_process + "/"
pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)
print("grandparent_directory:", grandparent_directory)
print("DataPath:", DataPath)
DataPath = DataPath + "data_size="+str(num_train)+"_deg="+str(deg)+"_e="+str(e)+"_d="+str(d)+"_coef_seed="+str(coef_seed)+"_diff_W/"
pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)

grandparent_directory: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Code_MacBook
DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Test0330_DDR_Data_Generation/


# Generate Data

In [ ]:
def Prepare_Data(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process,x_dist, e_dist, x_low, x_up, x_mean, x_var, bump):
# #  ****** Coef generation *********
    from Data import data_generation
    data_gen = data_generation()
    # W_star = data_gen.generate_truth(DataPath,lower, upper, p, d, coef_seed,version = 0) 
    # print("W_star = ",W_star[0,:])
    np.random.seed(coef_seed)
    x_test_all = {}; c_test_all = {}; x_train_all = {}; c_train_all = {}; W_star_all = {}
    for iter in iteration_all:
        W_star = data_gen.generate_truth(DataPath,lower, upper, p, d, iter,version = 0) 
        DataPath_seed = DataPath +"iter="+str(iter)+"/"
        pathlib.Path(DataPath_seed).mkdir(parents=True, exist_ok=True)
        # #  ****** Data generation *********
        x_test_all[iter], c_test_all[iter], x_train_all[iter], c_train_all[iter], W_star_all[iter] = data_gen.generate_samples(iter,DataPath_seed,p, d, num_test, num_train, alpha, W_star, mis, num_test, 
                                data_generation_process, x_dist, e_dist, x_low, x_up, x_mean, x_var, bump) 
        # print()
    return x_test_all, c_test_all, x_train_all, c_train_all, W_star_all

# EPO(SPO+,...)

In [6]:
# from PYEPO import PyEPO_Method
# epo_runner = PyEPO_Method()
def Implement_EPO(DataPath,seed_all,batch_size,num_epochs,method_names,x_train_all,c_train_all,x_test_all,c_test_all,arcs,epo_runner):
    cost_EPO = {}
    for seed in seed_all:
        DataPath_seed = DataPath +"Seed="+str(seed)+"/"
        pathlib.Path(DataPath_seed).mkdir(parents=True, exist_ok=True)
        # print("*** seed = ",seed,": Run EPO ******")
        cost_EPO[seed] = epo_runner.run(method_names,DataPath_seed,batch_size,num_feat,grid,num_epochs,\
                                        x_train_all[seed],c_train_all[seed],x_test_all[seed],c_test_all[seed],arcs)
    return cost_EPO

# Obtain DDR estimation

### define network

In [7]:
def _getArcs(grid):
    arcs = []
    for i in range(grid[0]):
        # edges on rows
        for j in range(grid[1] - 1):
            v = i * grid[1] + j
            arcs.append((v, v + 1))
        # edges in columns
        if i == grid[0] - 1:
            continue
        for j in range(grid[1]):
            v = i * grid[1] + j
            arcs.append((v, v + grid[1]))

    arc_index_mapping = {}
    for i in range(len(arcs)):
        arc = arcs[i]
        arc_index_mapping[arc] = i

    return arcs,arc_index_mapping

In [8]:
def solve_DDR(arcs,lamb,mu_fixed,num_nodes,x_train,c_train):
    
    N,p = x_train.shape
    N,d = c_train.shape

    # DDR
    m = Model("ddr")
    #m.setParam("DualReductions",0)
    m.setParam('OutputFlag', 0)

    W_ind = tuplelist( [(i,j) for i in range(d) for j in range(p)] )
    w0_ind = tuplelist( [i for i in range(d)])

    W_ddr = m.addVars(W_ind, lb=-GRB.INFINITY,name = "W" )
    w0_ddr = m.addVars(w0_ind, lb=-GRB.INFINITY,name = "W0" )
    alpha = m.addVars(N,num_nodes,lb=-GRB.INFINITY,name="alpha")
    expr_obj = 0
    err = []
    for n in range(N):
        cost_true_tem = c_train[n]
        expr_obj = expr_obj + alpha[n,num_nodes-1] - alpha[n,0]
        for ind in range(len(arcs)):
            cost_pred_tem = quicksum([W_ddr[ind,j] * x_train[n,j] for j in range(p)]) + w0_ddr[ind]
            err.append(cost_true_tem[ind] - cost_pred_tem)
            e = arcs[ind]
            j = e[1]
            i = e[0]
            # print("j = ",j,", i = ",i, ", e = ",e)
            m.addConstr(alpha[n,j] - alpha[n,i] >= -mu_fixed*cost_true_tem[ind] - (1-mu_fixed)*cost_pred_tem)

    m.setObjective(quicksum([err[k] * err[k] for k in range(len(err))])/N + lamb*(expr_obj)/N, GRB.MINIMIZE)
    m.optimize()
    
    W_DDR_rst = m.getAttr('x', W_ddr)
    w0_DDR_rst = m.getAttr('x', w0_ddr)
    W_ddr_val = []
    for i in range(d):
        W_ddr_val.append([W_DDR_rst[(i,j)] for j in range(p)])
    w0_ddr_val = [w0_DDR_rst[i] for i in range(d)]

    alpha_rst = m.getAttr('x', alpha)
    return w0_ddr_val,W_ddr_val,alpha_rst,m.ObjVal

# Out-of-sample performance

### Generate data

In [ ]:
arcs,arc_index_mapping = _getArcs(grid)
num_arcs = len(arcs)
iteration_all = np.arange(0,1)
# obtain data
x_test_all, c_test_all, x_train_all, c_train_all, W_star_all = Prepare_Data(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process)

### EPO performance

In [10]:
# # EPO, including SPO, PG, LTR
# batch_size = 20
# num_epochs = 100
# method_names = ["spo+","pg","ltr"]
# method_names = ["spo+"]
# from PYEPO import PyEPO_Method
# epo_runner = PyEPO_Method()

# cost_EPO_all = Implement_EPO(DataPath,[1],batch_size,num_epochs,method_names,x_train_all,c_train_all,x_test_all,c_test_all,arcs,epo_runner)
# print("SPO+Cost = ",np.nanmean(cost_EPO_all[1]["SPO"]))

## Oracle performance

In [ ]:
from Peformance import performance_evaluation
perfs = performance_evaluation()
cost_Oracle_all = {}
for iter in iteration_all:
    # compute OLS performance
    cost_Oracle_all[iter] = perfs.compute_Cost_with_Prediction(arcs,W_star_all[iter],np.ones(num_arcs)*bump, grid,c_test_all[iter],x_test_all[iter])
    # print("Oracle Cost = ",np.nanmean(cost_Oracle_all[iter]))

### OLS performance

In [ ]:
from OLS import ols_method
ols_method_obj = ols_method()
W_ols_all = {}; w0_ols_all = {}; t_ols_all = {}; obj_ols_all = {}
cost_OLS_all = {}
for iter in iteration_all:
    # compute OLS performance
    W_ols_all[iter], w0_ols_all[iter], t_ols_all[iter], obj_ols_all[iter] = ols_method_obj.ols_solver("",x_train_all[iter], c_train_all[iter])
    cost_OLS_all[iter] = perfs.compute_Cost_with_Prediction(arcs,w0_ols_all[iter],W_ols_all[iter], grid,c_test_all[iter],x_test_all[iter])
    print("OLS cost = ",np.nanmean(cost_OLS_all[iter]))

Set parameter Username
Academic license - for non-commercial use only - expires 2026-03-13
Oracle Cost =  199.07499772359128 ,OLS cost =  206.81770560425903
Oracle Cost =  199.64237741065355 ,OLS cost =  207.23339668979605
Oracle Cost =  199.33044121318946 ,OLS cost =  206.61422311815087
Oracle Cost =  199.86208532050733 ,OLS cost =  207.5373436676744
Oracle Cost =  199.11624056821543 ,OLS cost =  206.19173020396562
Oracle Cost =  199.3396421072896 ,OLS cost =  207.38384154331126
Oracle Cost =  198.77775968948427 ,OLS cost =  206.33038454230942
Oracle Cost =  199.4237022531088 ,OLS cost =  206.96668482735214
Oracle Cost =  200.1233913465645 ,OLS cost =  207.94452419517393
Oracle Cost =  200.07344662197266 ,OLS cost =  207.0080396250465
Oracle Cost =  199.1754623477314 ,OLS cost =  206.71363085744747
Oracle Cost =  199.0808494151275 ,OLS cost =  206.9805734317515
Oracle Cost =  200.09933551587125 ,OLS cost =  206.79709899643473
Oracle Cost =  199.46594064627746 ,OLS cost =  206.66500949

### DDR performance

In [12]:
def obtain_DDR_out_of_sample_performance(mu_all,lamb_all,num_nodes,x_train,c_train,x_test,c_test,perfs):
    lamb_index = 0
    cost_DDR = {}; w0_ddr_dict = {}; W_ddr_dict = {}
    cost_DDR_avg = np.zeros((len(mu_all),len(lamb_all)))
    mu_index = 0
    for mu in mu_all:
        lamb_index = 0
        for lamb in lamb_all:
            w0_ddr_dict[mu,lamb],W_ddr_dict[mu,lamb],alpha_rst,obj_ddr = solve_DDR(arcs,lamb,mu,num_nodes,x_train,c_train)
            cost_DDR[mu,lamb] = perfs.compute_Cost_with_Prediction(arcs,w0_ddr_dict[mu,lamb],W_ddr_dict[mu,lamb], grid,c_test,x_test)
            cost_DDR_avg[mu_index,lamb_index] = np.nanmean(cost_DDR[mu,lamb])
            lamb_index = lamb_index + 1
        # print("cost_DDR_avg=",np.round(cost_DDR_avg[0,:],4))
        mu_index = mu_index + 1
    return cost_DDR,w0_ddr_dict,W_ddr_dict,cost_DDR_avg

In [ ]:
mu_all = np.arange(0.01,1.0,0.1)
lamb_all = np.round(np.arange(0.01,1,0.1),4)
regret_all = {}; cost_DDR_all = {}; w0_ddr_all = {}; W_ddr_all = {}
cost_DDR_avg_all = np.zeros((len(mu_all),len(lamb_all)))
regred_DDR_vs_OLS_all = np.zeros((len(mu_all),len(lamb_all)))

for iter in iteration_all:
    cost_DDR_all[iter],w0_ddr_all[iter],W_ddr_all[iter],cost_DDR_avg = obtain_DDR_out_of_sample_performance(mu_all,lamb_all,num_nodes,x_train_all[iter],c_train_all[iter],x_test_all[iter],c_test_all[iter],perfs)
    cost_DDR_avg_all = cost_DDR_avg_all + cost_DDR_avg
    regert_tem = (np.nanmean(cost_OLS_all[iter]) - cost_DDR_avg)/(np.nanmean(cost_OLS_all[iter]) - np.nanmean(cost_Oracle_all[iter])) * 100
    print("iter=",iter,",DDR_vs_OLS = ",regert_tem)
    regred_DDR_vs_OLS_all = regred_DDR_vs_OLS_all + regert_tem
cost_DDR_avg_all = cost_DDR_avg_all/len(iteration_all)
regred_DDR_vs_OLS_all = regred_DDR_vs_OLS_all/len(iteration_all)
print("regred_DDR_vs_OLS_all=",regred_DDR_vs_OLS_all)

iter= 1 ,DDR_vs_OLS =  [[-0.34317672 -0.25203866  0.38034829 -0.37285252 -0.38112009 -0.18465626
  -0.67522638 -1.36663584 -2.11806747]
 [ 0.35300222 -0.54128809  0.1897131  -1.50885584 -1.7631667  -2.21324509
  -1.59512994 -2.59977472 -2.10046848]
 [-0.13920288 -0.78167331 -0.48541047 -1.14514464 -2.71685395 -2.43287072
  -3.08833289 -3.01397589 -2.7511866 ]
 [ 0.         -0.13920288 -0.62941144 -0.64069952 -0.64771456 -1.14514464
  -2.16805357 -2.05765318 -2.77557176]
 [ 0.          0.         -0.13920288  0.23199322  0.23199322  0.53264538
   0.53264538 -0.32745154 -0.33162904]]
iter= 2 ,DDR_vs_OLS =  [[-0.13327869 -0.34201543 -0.29374848 -1.2149897  -1.31960252 -2.77085059
  -2.13137443 -2.6351975  -2.88917432]
 [-0.38320654 -0.34201543 -0.20025777 -0.29374848 -1.2149897  -0.90204321
  -1.95743801 -2.52366189 -2.02468269]
 [-0.15585733 -0.36459407 -0.36459407 -1.83252009 -2.5072043  -1.89500242
  -2.80664547 -3.00392901 -2.04592705]
 [-0.37732175 -0.15585733 -0.15585733 -0.36459407

In [24]:
print("regred_DDR_vs_OLS_all=",np.round(regred_DDR_vs_OLS_all,4))

regred_DDR_vs_OLS_all= [[-0.0293 -0.0503  0.044  -0.0441  0.0913  0.0206  0.1666  0.1237 -0.2018]
 [-0.1015 -0.2112 -0.0081 -0.2691 -0.3541 -0.2353 -0.1575 -0.2446  0.0422]
 [-0.0981 -0.2561 -0.2737 -0.3017 -0.4145 -0.4586 -0.5319 -0.4253 -0.3679]
 [-0.1186 -0.1317 -0.1012 -0.1715 -0.2131 -0.271  -0.1845 -0.2076 -0.3549]
 [-0.1056 -0.0224  0.0506  0.0679 -0.0014  0.0579 -0.002  -0.0168 -0.0119]]


In [15]:
# with open(DataPath+'cost_OLS_all.pkl', "wb") as tf:
#     pickle.dump(cost_OLS_all,tf)
# with open(DataPath+'cost_Oracle_all.pkl', "wb") as tf:
#     pickle.dump(cost_Oracle_all,tf)
# with open(DataPath+'cost_DDR_all.pkl', "wb") as tf:
#     pickle.dump(cost_DDR_all,tf)

In [16]:
# seed = 1
# lamb = 1
# cost_OLS_esti = x_test_all[seed] @ np.asarray(W_ols_all[seed]).T + w0_ols_all[seed]
# cost_DDR_esti = x_test_all[seed] @ np.asarray(W_ddr_all[seed,mu][lamb]).T + w0_ddr_all[seed,mu][lamb]

# S_test_index = 3
# print("cost_DDR_esti=",np.round(cost_DDR_esti[S_test_index,],2))
# # print("cost:(0,1)-->(1,3) = ",cost_DDR_esti[S_test_index,0] + cost_DDR_esti[S_test_index,2])
# # print("cost:(0,2)-->(2,3) = ",cost_DDR_esti[S_test_index,1] + cost_DDR_esti[S_test_index,3])

# print("cost_OLS_esti=",np.round(cost_OLS_esti[S_test_index,],2))
# # print("cost:(0,1)-->(1,3) = ",cost_OLS_esti[S_test_index,0] + cost_OLS_esti[S_test_index,2])
# # print("cost:(0,2)-->(2,3) = ",cost_OLS_esti[S_test_index,1] + cost_OLS_esti[S_test_index,3])


# print("cost_Oracle=",np.round(c_test_all[seed][S_test_index],2))
# # print("cost:(0,1)-->(1,3) = ",c_test_all[seed][S_test_index,0] + c_test_all[seed][S_test_index,2])
# # print("cost:(0,2)-->(2,3) = ",c_test_all[seed][S_test_index,1] + c_test_all[seed][S_test_index,3])

# Plot figures

In [17]:
def cross_compare2plus(c_item, c_base, c_oracle):
    c_item = np.asarray(c_item)
    c_base = np.asarray(c_base)
    c_oracle = np.asarray(c_oracle)

    N = len(c_item)
    c_diff = c_item - c_base
    lbel = np.zeros((N,1))
    
    equals = np.sum(c_diff == 0)
    wins = np.sum(c_diff < 0)
    lose = np.sum(c_diff > 0)
    
    lbel[c_diff < 0] = 1
    lbel[c_diff > 0] = -1
    
    # print("Num_train =",N,",Num_equals =",equals,",Num_wins =",wins,",Num_lose =",lose)
    # print("base cost = ", np.mean(c_base),",item cost = ",np.mean(c_item))
    if N == equals:
        win_ratio = 0.5
    else:
        win_ratio = wins/(N - equals)
    # cost_reduction = (np.mean(c_base) - np.mean(c_item) )/np.abs(np.mean(c_oracle))
    regret_reduction = (np.nanmean(c_base) - np.nanmean(c_item))/np.abs(np.nanmean(c_base) - np.nanmean(c_oracle))
    return lbel, win_ratio, regret_reduction

### DDR figure

In [18]:
# H2H_DDR_vs_OLS_all = {}; regret_reduction_DDR_vs_OLS_all = {}
# for mu in mu_all:
#     for lamb in lamb_all:
#         H2H_DDR_vs_OLS_arr = np.zeros(len(iteration_all)); regret_reduction_DDR_vs_OLS_arr = np.zeros(len(iteration_all))
#         # print("lamb = ",lamb)
#         for iter in iteration_all:
#             lbel, H2H_DDR_vs_OLS_arr[iter-1], regret_reduction_DDR_vs_OLS_arr[iter-1] = cross_compare2plus(cost_DDR_all[iter][mu,lamb],cost_OLS_all[iter], cost_Oracle_all[iter])
#         H2H_DDR_vs_OLS_all[mu,lamb] = H2H_DDR_vs_OLS_arr; regret_reduction_DDR_vs_OLS_all[mu,lamb] = regret_reduction_DDR_vs_OLS_arr

In [19]:
# mu = mu_all[0]
# lamb = lamb_all[7]
# import Figures
# file_name = DataPath + "figure_mu="+str(mu)+"_lamb="+str(lamb)
# Figures.figure_plot_upleft(H2H_DDR_vs_OLS_all[mu,lamb]*100, regret_reduction_DDR_vs_OLS_all[mu,lamb], file_name, size = (5, 5), move = [-0.12, 0.04, 0.35, 0.55], 
#                     ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

In [20]:
# import matplotlib.pyplot as plt
# fig = plt.figure()
# ax = plt.subplot()
# ax.plot(lamb_all,regret_all[seed,mu])

### SPO figure

In [21]:
# H2H_SPO_vs_OLS_arr = np.zeros(len(seed_all)); regret_reduction_SPO_vs_OLS_arr = np.zeros(len(seed_all))
# seed_index = 0
# for seed in seed_all:
#     lbel, H2H_SPO_vs_OLS_arr[seed_index], regret_reduction_SPO_vs_OLS_arr[seed_index] = cross_compare2plus(cost_EPO_all[seed]["SPO"],cost_OLS_all[seed], cost_Oracle_all[seed])
#     seed_index = seed_index + 1
# H2H_SPO_vs_OLS_all = H2H_SPO_vs_OLS_arr; regret_reduction_SPO_vs_OLS_all = regret_reduction_SPO_vs_OLS_arr

In [22]:
# Figures.figure_plot_upleft(H2H_SPO_vs_OLS_all*100, regret_reduction_SPO_vs_OLS_all, "", size = (5, 5), move = [-0.12, 0.04, 0.35, 0.55], 
#                     ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

In [23]:
# regret_reduction_SPO_vs_OLS_arr